This will be mainly use for capstone project

In [4]:
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import json
try:
    import folium
except:
    !pip install folium
    import folium
import matplotlib.pyplot as mplpy
import seaborn as sns
import matplotlib.colors as color
import matplotlib.cm as cm
from geopy.geocoders import Nominatim
import warnings
warnings.simplefilter('ignore')
from bs4 import BeautifulSoup
try:
    import pgeocode
except:
    !pip install pgeocode
    import pgeocode
print('Library imported')

Library imported


In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).text
soup= BeautifulSoup(result,'html5lib')
table = soup.find_all('table')

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [7]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df.shape

(103, 3)

In [9]:
latitude=[]
longitude=[]
for postcode in df['PostalCode']:
    nomi = pgeocode.Nominatim('CA')
    data = nomi.query_postal_code(postcode)
    latitude.append(data['latitude'])
    longitude.append(data['longitude'])
df['Latitude']=latitude
df['Longitude']=longitude

In [23]:
df.dropna(inplace=)

0      43.7545
1      43.7276
2      43.6555
3      43.7223
4      43.6641
        ...   
98     43.6518
99     43.6656
100    43.7804
101    43.6325
102    43.6256
Name: Latitude, Length: 102, dtype: float64

In [11]:
add= 'Toronto,CA'
geolocation = Nominatim(user_agent='nt_explorer')
location = geolocation.geocode(add)
latitude = location.latitude
longitude = location.longitude

In [14]:
toronto_map = folium.Map([latitude,longitude],zoom_start=10)
for lat,long,postcod,borough in zip(df['Latitude'],df['Longitude'],df['PostalCode'],df['Borough']):
    folium.CircleMarker( [lat,long],
                        popup = ('PCode: '+str(postcod)+'<br>' 'Borough: '+str(borough)+'<br>'),
                          color='blue',
                          fill_color='blue',
                          fill_capacity=0.7,
                          parse_html=False
                        ).add_to(toronto_map)
toronto_map

ValueError: Location values cannot contain NaNs.